# Moving data from Postgres to SQL Server

Start the databases

Run this from the terminal:

```
docker-compose up
```

Let's connect to Postgres and select some data

In [ ]:
import psycopg2

pg_host = 'localhost'
pg_port = 5432
pg_user = 'postgres'
pg_pass = 'postgres'
pg_db = 'pgdb'

pg_conn = psycopg2.connect(host=pg_host,user=pg_user,password=pg_pass,dbname=pg_db)
cur = pg_conn.cursor()
cur.execute("select * from movies")
movies = cur.fetchall()
print(movies)
pg_conn.close()

Let's load from Postgres into a data frame

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')
pg_conn = engine.connect()

#df = pd.read_csv("https://raw.githubusercontent.com/kirenz/datasets/master/gapminder.csv")
#df.to_csv('data/csv/gapminder.csv', mode='w') # overwrite
df = pd.read_csv('./data/csv/gapminder.csv')
df.to_sql('gapminder', pg_conn, if_exists='replace')
pg_conn.commit()

df2 = pd.read_sql_query(sql=text('select * from gapminder'), con=pg_conn)
print(df2)

pg_conn.close()


Let's load from SQL Server

In [ ]:
import pyodbc

# create database if it doesn't exist

ms_host = 'localhost'
ms_port = 1433
ms_user = 'sa'
ms_pass = 'Password123'
ms_db = 'Movies'

mssql_conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};'
                            f'Server={ms_host};'
                            'Database=master;'
                            f'uid={ms_user};pwd={ms_pass};'
                            'TrustServerCertificate=yes;',
                            autocommit=True)
mssql_conn.execute("IF NOT EXISTS(SELECT * FROM sys.databases WHERE name = 'Movies') BEGIN CREATE DATABASE Movies END")
mssql_conn.commit()
mssql_conn.close()

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

ms_host = 'localhost'
ms_port = 1433
ms_user = 'sa'
ms_pass = 'Password123'
ms_db = 'Movies'

# connect to the db
mssql_engine = create_engine(f'mssql+pyodbc://{ms_user}:{ms_pass}@{ms_host}/{ms_db}?driver=SQL+Server+Native+Client+11.0&TrustServerCertificate=yes')
mssql_conn = mssql_engine.connect()

# load data into database
df = pd.read_csv('./data/csv/gapminder.csv')
df.to_sql('GapMinder', mssql_engine, schema='dbo', if_exists='replace')
mssql_conn.commit()

# select data from database
df3 = pd.read_sql_query(sql=text('select * from dbo.GapMinder'), con=mssql_conn)
print(df3)

mssql_conn.close()


Now that we've successfully connected to each database, let's move data from Postgres to SQL Server

In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

# get data from Postgres
pg_engine = create_engine(f'postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')
pg_conn = pg_engine.connect()

df = pd.read_sql_query(sql=text('select * from movies'), con=pg_conn)

pg_conn.close()

# save data to SQL Server
mssql_engine = create_engine(f'mssql+pyodbc://{ms_user}:{ms_pass}@{ms_host}/{ms_db}?driver=SQL+Server+Native+Client+11.0&TrustServerCertificate=yes')
mssql_conn = mssql_engine.connect()

mssql_conn.execute(statement=text('drop table if exists dbo.Movies'))
df.to_sql('Movies', mssql_engine, schema='dbo', if_exists='replace')

df2 = pd.read_sql_query(sql=text('select * from dbo.Movies'), con=mssql_conn)

mssql_conn.commit()
mssql_conn.close()

What data did we load?

In [ ]:
df2